# Imports

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, ZeroPadding2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.models import Model

## 1. Comparación: Capas Convolucionales vs Capas Densas

Una de las principales ventajas de las redes convolucionales es su **eficiencia en parámetros**. Vamos a comparar cuántos parámetros necesita una capa convolucional frente a una capa densa para procesar la misma entrada.

In [ ]:
input = Input(shape=(27, 27, 1))
x = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1,1))(input)

print(f"Entrada: {input.shape}")
print(f"Salida: {x.shape}")
print(f"Filtros: 64, Kernel: 3x3, Padding: same, Stride: 1")

In [ ]:
model_conv = Model(input, x)
model_conv.summary()
print(f"\nPesos del filtro: 3 × 3 × 1 × 64 = {3*3*1*64:,} parámetros")
print(f"Bias: 64 parámetros") 
print(f"Total: {3*3*1*64 + 64:,} parámetros")

In [ ]:
input = Input(shape=(27, 27, 1))
x = Flatten()(input)  # Convierte (27, 27, 1) → (729,)
x = Dense(64)(x)      # 64 neuronas de salida

print(f"Entrada original: (27, 27, 1)")
print(f"Después de Flatten: {x.shape[1]} elementos")
print(f"Salida Dense: {x.shape}")
print(f"Neuronas: 64")

In [ ]:
model_dense = Model(input, x)
print("PARÁMETROS CAPA DENSA:")
model_dense.summary()
print(f"\nPesos: 729 × 64 = {729*64:,} parámetros")
print(f"Bias: 64 parámetros")
print(f"Total: {729*64 + 64:,} parámetros")

In [ ]:
print(f"Capa Convolucional: {3*3*1*64 + 64:,} parámetros")
print(f"Capa Densa: {729*64 + 64:,} parámetros")
print(f"Ratio: {((729*64 + 64) / (3*3*1*64 + 64)):.1f}x más parámetros en la capa densa")

## 2. Efectos del Padding

Vamos a ver cómo el tipo de padding afecta las dimensiones de salida:

In [ ]:
# PADDING = 'SAME' (mantiene dimensiones)
input = Input(shape=(28, 28, 1))

# Diferentes tamaños de kernel con padding='same'
kernels = [3, 5, 7]
for k in kernels:
    x = Conv2D(filters=32, kernel_size=(k, k), padding='same', strides=1)(input)
    print(f"Kernel {k}x{k}: {input.shape} → {x.shape}")
    
print(f"\n Con padding='same' y stride=1, las dimensiones se mantienen iguales")

In [ ]:
# PADDING = 'VALID' (reduce dimensiones)
input = Input(shape=(28, 28, 1))

for k in kernels:
    x = Conv2D(filters=32, kernel_size=(k, k), padding='valid', strides=1)(input)
    reduction = 28 - x.shape[1]
    print(f"Kernel {k}x{k}: {input.shape} → {x.shape} (reducción: {reduction} píxeles)")

print(f"\n Con padding='valid', se pierden píxeles en los bordes")
print(f"Fórmula: output_size = input_size - kernel_size + 1")

## 3. Efectos del Stride (Paso de la Convolución)

El stride controla cuánto **downsampling** (reducción de dimensiones) se produce:

In [ ]:
# COMPARACIÓN DE DIFERENTES STRIDES
input = Input(shape=(32, 32, 3))  # Imagen RGB típica

strides = [1, 2, 3, 4]
for s in strides:
    x = Conv2D(filters=64, kernel_size=3, padding='same', strides=s)(input)
    reduction_factor = input.shape[1] / x.shape[1]
    print(f"Stride {s}: {input.shape} → {x.shape} (factor reducción: {reduction_factor:.1f}x)")

print(f"\n Regla: Con padding='same', output_size = input_size / stride")

In [ ]:
# STRIDE CON PADDING='VALID'
input = Input(shape=(28, 28, 1))

for s in [1, 2, 3]:
    x = Conv2D(filters=32, kernel_size=5, padding='valid', strides=s)(input)
    # Fórmula: (input_size - kernel_size + 1) / stride
    expected = (28 - 5 + 1) // s
    print(f"Stride {s}: {input.shape} → {x.shape} (esperado: {expected})")

print(f"\n Stride alto con padding='valid' puede reducir drásticamente las dimensiones")

## 5. Múltiples Convoluciones Anidadas

En redes reales, las convoluciones se **apilan** para crear jerarquías de características. Veamos cómo evolucionan las dimensiones a través de múltiples capas:

In [ ]:
# RED CONVOLUCIONAL TÍPICA (como LeNet/VGG)

input = Input(shape=(224, 224, 3))  # ImageNet input
x = input

# Bloque 1
x = Conv2D(8, (3, 3), padding='same', name='conv1_1')(x)
print(f"Conv1_1: {x.shape}")
x = Conv2D(16, (3, 3), padding='same', name='conv1_2')(x) 
print(f"Conv1_2: {x.shape}")
print()

# Embudo
print("También MaxPooling2D para reducir dimensiones, pero aquí usamos Conv2D con stride 2")
x = Conv2D(32, (3, 3), padding='same', name='pool1', strides=(3, 3))(x)
print(f"Reducción1: {x.shape}")
x = Conv2D(64, (3, 3), padding='same', name='pool2', strides=(3, 3))(x)
print(f"Reducción2: {x.shape}")
print()

# Densa
x = Flatten()(x)
x = Dense(16, activation='relu', name='fc')(x)
print(f"Densa: {x.shape}")
x = Dense(3, activation='softmax', name='output')(x) # 3 clases
print(f"Salida: {x.shape}")
model = Model(input, x)
print("\nRESUMEN DEL MODELO COMPLETO:")
model.summary()

## Arquitectura muy usada en CNN

NOTA: Veremos qué son las capas MaxPooling

![VGG](https://viso.ai/wp-content/uploads/2024/04/vgg-16.bak-1280x708.png)

---

Creado por **Guillermo Iglesias** (guillermo.iglesias@upm.es) y **Jorge Dueñas Lerín** (jorge.duenas.lerin@upm.es)

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png">